In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_extraction.text import TfidfVectorizer
#função para pré processamento de textos

from nltk.corpus import stopwords
import nltk
import tqdm

#Importar dataset Kaggle

In [ ]:
!pip3 install -q kaggle

In [ ]:
kjson = pd.read_json('kaggle.json', typ='series')

In [ ]:
os.environ['KAGGLE_USERNAME']= "emillyvictoria"
os.environ['KAGGLE_KEY']= kjson.key

In [ ]:
!kaggle datasets download --force tboyle10/medicaltranscriptions

 21% 1.00M/4.85M [00:00<00:00, 7.36MB/s]
100% 4.85M/4.85M [00:00<00:00, 21.9MB/s]


In [ ]:
!unzip medicaltranscriptions.zip

Archive:  medicaltranscriptions.zip
replace mtsamples.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

#Leitura inicial dos dados

In [ ]:
#FOCO: utilizando a transcription vamos prever a especialidade do médico

In [ ]:
df = pd.read_csv('/content/mtsamples.csv')
df.head()

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


In [ ]:
df.isnull().sum()
#valores faltantes

Unnamed: 0              0
description             0
medical_specialty       0
sample_name             0
transcription          33
keywords             1068
dtype: int64

#Pré processamento

In [ ]:
df['medical_specialty'].value_counts()

 Surgery                          1103
 Consult - History and Phy.        516
 Cardiovascular / Pulmonary        372
 Orthopedic                        355
 Radiology                         273
 General Medicine                  259
 Gastroenterology                  230
 Neurology                         223
 SOAP / Chart / Progress Notes     166
 Obstetrics / Gynecology           160
 Urology                           158
 Discharge Summary                 108
 ENT - Otolaryngology               98
 Neurosurgery                       94
 Hematology - Oncology              90
 Ophthalmology                      83
 Nephrology                         81
 Emergency Room Reports             75
 Pediatrics - Neonatal              70
 Pain Management                    62
 Psychiatry / Psychology            53
 Office Notes                       51
 Podiatry                           47
 Dermatology                        29
 Cosmetic / Plastic Surgery         27
 Dentistry               

In [ ]:
df = df.groupby("medical_specialty").filter(lambda x: len(x) > 200)
#retira todas as especialidades menores que 200
#poderia ser usada também 
#df funcao qualquer (x):
#  return len(x) > 200

#Removendo NaNs

In [ ]:
df= df[['transcription', 'medical_specialty']]
df = df.drop(df[df['transcription'].isna()].index)

In [ ]:
df.reset_index(drop=True, inplace=True)

#Text Process

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stopwords.words('english')
#as stopwords são, na maioria das vezes, palavras irrelevantes para o modelo

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [ ]:
stopwords.words('portuguese')

['de',
 'a',
 'o',
 'que',
 'e',
 'é',
 'do',
 'da',
 'em',
 'um',
 'para',
 'com',
 'não',
 'uma',
 'os',
 'no',
 'se',
 'na',
 'por',
 'mais',
 'as',
 'dos',
 'como',
 'mas',
 'ao',
 'ele',
 'das',
 'à',
 'seu',
 'sua',
 'ou',
 'quando',
 'muito',
 'nos',
 'já',
 'eu',
 'também',
 'só',
 'pelo',
 'pela',
 'até',
 'isso',
 'ela',
 'entre',
 'depois',
 'sem',
 'mesmo',
 'aos',
 'seus',
 'quem',
 'nas',
 'me',
 'esse',
 'eles',
 'você',
 'essa',
 'num',
 'nem',
 'suas',
 'meu',
 'às',
 'minha',
 'numa',
 'pelos',
 'elas',
 'qual',
 'nós',
 'lhe',
 'deles',
 'essas',
 'esses',
 'pelas',
 'este',
 'dele',
 'tu',
 'te',
 'vocês',
 'vos',
 'lhes',
 'meus',
 'minhas',
 'teu',
 'tua',
 'teus',
 'tuas',
 'nosso',
 'nossa',
 'nossos',
 'nossas',
 'dela',
 'delas',
 'esta',
 'estes',
 'estas',
 'aquele',
 'aquela',
 'aqueles',
 'aquelas',
 'isto',
 'aquilo',
 'estou',
 'está',
 'estamos',
 'estão',
 'estive',
 'esteve',
 'estivemos',
 'estiveram',
 'estava',
 'estávamos',
 'estavam',
 'estivera'

In [ ]:
import string

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def processamento(texto):
  STOPWORDS= stopwords.words('english') + ['ur','im', 'dont', 'doin']

  #cria uma lista dos caracteres excluindo pontuação
  sem_pontuacao = [char for char in texto if char not in string.punctuation]

  #junta os caracteres da lista formada
  sem_pontuacao= ''.join(sem_pontuacao)

  #remove as stopwords
  return ' '.join(palavra for palavra in sem_pontuacao.split() if palavra.lower() not in STOPWORDS)
#RESUMINDO: pega o texto e retira as pontuações; coloca tudo em minúsculo: confere se tem stopword; se tiver, ele retira

In [ ]:
df ['transcription'] = df['transcription'].apply(processamento)
df ['transcription']
#passando a função acima para todas linhas

0       2D MMODE 1 Left atrial enlargement left atrial...
1       1 left ventricular cavity size wall thickness ...
2       2D ECHOCARDIOGRAMMultiple views heart great ve...
3       DESCRIPTION1 Normal cardiac chambers size2 Nor...
4       2D STUDY1 Mild aortic stenosis widely calcifie...
                              ...                        
3304    INDICATION Chest painTYPE TEST Adenosine nucle...
3305    CHIEF COMPLAINT Chest painHISTORY PRESENT ILLN...
3306    HISTORY PRESENT ILLNESS patient 68yearold woma...
3307    HISTORY PRESENT ILLNESS Mr ABC 60yearold gentl...
3308    REASON CONSULTATION Abnormal echocardiogram fi...
Name: transcription, Length: 3309, dtype: object

In [ ]:
df.head()

,transcription,medical_specialty
0,2D MMODE 1 Left atrial enlargement left atrial...,Cardiovascular / Pulmonary
1,1 left ventricular cavity size wall thickness ...,Cardiovascular / Pulmonary
2,2D ECHOCARDIOGRAMMultiple views heart great ve...,Cardiovascular / Pulmonary
3,DESCRIPTION1 Normal cardiac chambers size2 Nor...,Cardiovascular / Pulmonary
4,2D STUDY1 Mild aortic stenosis widely calcifie...,Cardiovascular / Pulmonary


#Encoder

In [ ]:
y = pd.get_dummies(df.medical_specialty)
#retira uma das categorias
#1= yes | 0= false

In [ ]:
y.head()

,Cardiovascular / Pulmonary,Consult - History and Phy.,Gastroenterology,General Medicine,Neurology,Orthopedic,Radiology,Surgery
0,1,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0


#TDIDF - Term frequency inverse document frequency

In [ ]:
vec = TfidfVectorizer()
#countVectorizer: converte uma coleção de textos para uma matriz de tokens, ou seja, cria uma feature para cada palavra do dataset e o valor dessa coluna vai ser igual a quantas vezes ela apareceu no txto da linha
#Tfidf = countVectorizer + transformação --> coloca um peso em cada palavra dependendo de quantas vezes ela aparece no dataset inteiro
#ex: palavra 'livro' aparace em todos os textos e linhas do dataset, a partir disso, o tfdf atribui um peso baixo a palavra 'livro' pois ela aparece muito, com isso, percebe que palavras que aparecem pouco tem um peso muito maior pois são 'especiais'

In [ ]:
df.transcription

0       2D MMODE 1 Left atrial enlargement left atrial...
1       1 left ventricular cavity size wall thickness ...
2       2D ECHOCARDIOGRAMMultiple views heart great ve...
3       DESCRIPTION1 Normal cardiac chambers size2 Nor...
4       2D STUDY1 Mild aortic stenosis widely calcifie...
                              ...                        
3304    INDICATION Chest painTYPE TEST Adenosine nucle...
3305    CHIEF COMPLAINT Chest painHISTORY PRESENT ILLN...
3306    HISTORY PRESENT ILLNESS patient 68yearold woma...
3307    HISTORY PRESENT ILLNESS Mr ABC 60yearold gentl...
3308    REASON CONSULTATION Abnormal echocardiogram fi...
Name: transcription, Length: 3309, dtype: object

In [ ]:
df_vec = vec.fit_transform(df.transcription)

In [ ]:
vec.get_feature_names()[-10:]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['zygomawe',
 'zyloprim',
 'zymar',
 'zyprexa',
 'zyprexasocial',
 'zyrtec',
 'zyrtecallergies',
 'zyrtecsurgeries',
 'zyvox',
 'µiuml']

In [ ]:
len(vec.get_feature_names())
#tá grande dms

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


41520

In [ ]:
#adicionando parâmetros
vec = TfidfVectorizer(min_df=5, lowercase=True, ngram_range=(1,2))
#min: só leva em consideração palavras que aparecem no mínimo em 5 ex de textos

df_vec = vec.fit_transform(df.transcription)
len(vec.get_feature_names())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


34708

In [ ]:
columns = vec.get_feature_names()
values = df_vec.A

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
df_tfidf = pd.DataFrame(values, columns = columns)

In [ ]:
df_tfidf.mean()

00           0.000327
000          0.000184
001          0.000087
0014         0.000117
0025         0.000074
               ...   
zosyn        0.000241
zygoma       0.000147
zygomatic    0.000159
zyrtec       0.000655
zyrtec 10    0.000239
Length: 34708, dtype: float64

#Separação

In [ ]:
x = df.transcription

from sklearn.model_selection import train_test_split
Xtrain, xval, ytrain, yval = train_test_split(x,y , test_size = 0.2, random_state=42)

In [ ]:
vec = TfidfVectorizer(min_df=5, lowercase=True, ngram_range=(1,2))

Xtrain = vec.fit_transform(Xtrain)
xval = vec.transform(xval)

#Random Forest

In [ ]:
rf = RandomForestClassifier(n_jobs=-1)
rf.fit(Xtrain, ytrain)

NameError: ignored

In [ ]:
pred = rf.predict(xval)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(yval, pred))
#segundo o prof n teve uma predição boa (antiquado)

##BERT

In [ ]:
#faz o pré processamento de uma forma mais moderna
#vamos usá-lo para fazer uma 'tokenização', ou seja, a codificação de textos 

In [ ]:
!pip install transformers -q

In [ ]:
import torch
import transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [ ]:
model_class = transformers.DistilBertModel
tokenizer_class = transformers.DistilBertTokenizer
pretrained_weights = 'distilbert-base-uncased'

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [ ]:
tokenized = df['transcription'].apply((lambda x: tokenizer.encode(x[:200], add_special_tokens=True)))

In [ ]:
tokenized
#transformando o texto em códigos

In [ ]:
#transformando em um input para um modelo de machine learning ou deep learning

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

#esse código visualiza linha por linha para descobrir qual é a maior linha, ou seja, a maior lista de códigos
#o padded preenche espaços vazios com zeros para todas as linhas terem o mesmo shape 

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape
#visualiza oq é padded e oq não é

In [ ]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    features = model(input_ids, attention_mask=attention_mask)
#cria tensores com o padded e o attention_mask para transformar em features

In [ ]:
features

In [ ]:
features = features[0][:,0,:].numpy()

In [ ]:
y = df['encoded_labels']
X = features

In [ ]:
Xtrain, Xval, ytrain, yval = train_test_split(features, labels, random_state=42

In [ ]:
Xtrain.shape

#Logis